# Filtrado de datos
Se eliminan duplicados, filas sin sinópisis y/o sin géneros. Luego se seleccionan los 18 géneros más comunes.

In [ ]:
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Loading dataset
dataset = load_dataset("pkchwy/letterboxd-all-movie-data")

train = dataset['train']

df = pd.json_normalize(train)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

full_dump.jsonl:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/847209 [00:00<?, ? examples/s]

In [ ]:
# filter all
filtered_df =  df.drop_duplicates(subset=['title'])

zero_genres = filtered_df['genres'].apply(lambda x: len(x) if isinstance(x, (list, tuple)) else 0).eq(0)
filtered_df = filtered_df[~zero_genres]

zero = filtered_df['synopsis'].apply(lambda x: len(x) if isinstance(x, str) else 0).eq(0)
filtered_df = filtered_df[~zero]

top18 = [
    "Drama", "Documentary", "Comedy", "Animation",
    "Horror", "Romance", "Thriller", "Music",
    "Action", "Crime", "Family", "Adventure",
    "Fantasy", "Science Fiction", "Mistery", "History",
    "War", "Western"
]

mask = filtered_df["genres"].apply(lambda g: all(genre in top18 for genre in g))
filtered_df = filtered_df[mask].copy()
filtered_df = filtered_df[["synopsis", "genres"]]

# Creación de datasets finales (con variedad de clases)
Se separa el dataset original según su uso:

Dataset One-genre: solo películas asignadas a un único género.
Dataset Multi-genre: películas con más de un género asignado.

Para cada uno se hará a su vez un split 80-10-10





## One Genre

In [ ]:
one_genre_df = filtered_df[filtered_df["genres"].apply(lambda x: len(x) == 1)]
unique_genres = one_genre_df["genres"].apply(lambda x: x[0]).unique()

for genre in unique_genres:
  count = len(one_genre_df[one_genre_df["genres"].apply(lambda x: x[0] == genre)])
  if count < 1500:
    one_genre_df = one_genre_df[one_genre_df["genres"].apply(lambda x: x[0] != genre)]

unique_genres = one_genre_df["genres"].apply(lambda x: x[0]).unique()

selected_films = {}
for genre in unique_genres:
  selected_films[genre] = one_genre_df[one_genre_df["genres"].apply(lambda x: x[0] == genre)].sort_values(by="synopsis", ascending=False)[:1500]
  selected_films[genre]["genre"] = genre
  selected_films[genre].drop(columns=["genres"], inplace=True)

one_genre_df = pd.concat(selected_films.values())
print(len(one_genre_df))

19500


## Multigenre

In [ ]:
one_genre_df = filtered_df[filtered_df["genres"].apply(lambda x: len(x) == 1)]
unique_genres = one_genre_df["genres"].apply(lambda x: x[0]).unique()

# nos quedamos con mil muestras de cada genero para poder tener algo de variedad a la hora de aprender -> 1 genero: 18000 ejemplares
# al resto le damos la misma cantidad de ejemplares:
selected_films = {}
for genre in unique_genres:
  selected_films[genre] = one_genre_df[one_genre_df["genres"].apply(lambda x: x[0] == genre)].sort_values(by="synopsis", ascending=False)[:100]

multi_genre_df = pd.concat(selected_films.values())
print(f"1 genre real: {len(multi_genre_df)}/18000")

two_genre_df = filtered_df[filtered_df["genres"].apply(lambda x: len(x) == 2)]
multi_genre_df = pd.concat([multi_genre_df, two_genre_df.sort_values(by="synopsis", ascending=False)[:1700]], axis=0)

three_genre_df = filtered_df[filtered_df["genres"].apply(lambda x: len(x) == 3)]
multi_genre_df = pd.concat([multi_genre_df, three_genre_df.sort_values(by="synopsis", ascending=False)[:1700]], axis=0)

#four_genre_df = filtered_df[filtered_df["genres"].apply(lambda x: len(x) == 4)]
#multi_genre_df = pd.concat([multi_genre_df, four_genre_df.sort_values(by="synopsis", ascending=False)[:1700]], axis=0)

# multigenre df has the following distribution:
# 1 genre: 1800
# 2 genres: 1800
# 3 genres: 1800
#not used: 4 genres: 1800
print(len(multi_genre_df))

1 genre real: 1700/18000
5100


# Análisis contenido de las sinopsis (por si se puede reutilizar para: ¿Existe una correlación entre la longitud de la sinopsis y el género?)

In [ ]:
import numpy as np
import html
import nltk

from nltk.corpus import stopwords
import string

nltk.download("stopwords")
all_synopsis = filtered_df["synopsis"]
len_synopsis = filtered_df["synopsis"].fillna("").apply(len).tolist()
total_empty_sinopsis = len_synopsis.count(0)
print(f"Total empty Synopsis: {total_empty_sinopsis}")

mean_len_before_filtering = np.mean(len_synopsis)
print(f"Synopsis mean len with empty: {mean_len_before_filtering}")
filtered_df = filtered_df[filtered_df["synopsis"].fillna("").apply(len) > 0]
print(f"Filtered_df len: {len(filtered_df)}")

len_synopsis = filtered_df["synopsis"].apply(len).tolist()
mean_len_before_filtering = np.mean(len_synopsis)
print(f"Synopsis mean len without empty: {mean_len_before_filtering}")

stop_words = stopwords.words("english")

def preprocess_text(text: str) -> str:
    """Limpia antes de tokenizar."""
    text = text.lower()
    text = text.replace("\\\\n", " ")
    text = html.unescape(text)
    exclude = {'#', "'", "@"}
    text = ''.join(char for char in text if char not in string.punctuation or char in exclude)
    return text


In [ ]:
def extract_adj_adv(text):
    doc = nlp(text)
    return [token.lemma_.lower() for token in doc if token.pos_ in ("ADJ")]

In [ ]:
import spacy
from tqdm import tqdm
tqdm.pandas()

nlp = spacy.load("en_core_web_sm")

from collections import defaultdict, Counter

one_genre_df["modifiers"] = one_genre_df["synopsis"].progress_apply(extract_adj_adv)

genre_word_counts = defaultdict(Counter)

for _, row in one_genre_df.iterrows():
    genre_word_counts[row["genre"]].update(row["modifiers"])

genre_word_counts["Horror"].most_common(10)


Usando PMI

In [ ]:
from collections import defaultdict
import math

word_counts = defaultdict(int)
joint_counts = defaultdict(int)
genre_counts = defaultdict(int)
total_films = len(one_genre_df)

for _, row in one_genre_df.iterrows():
    genre = row["genre"]
    genre_counts[genre] += 1
    for w in row["modifiers"]:
        word_counts[w] += 1
        joint_counts[(genre, w)] += 1

def calculate_pmi(genre, word):
    p_genre = genre_counts[genre] / total_films
    p_word = word_counts[word] / total_films
    p_joint = joint_counts[(genre, word)] / total_films

    if p_genre == 0 or p_word == 0 or p_joint == 0:
        return 0.0  # Avoid division by zero or log(0)
    return math.log2(p_joint / (p_genre * p_word))

pmi_results = defaultdict(list)
for genre in genres:
    for word in word_counts:
        pmi = calculate_pmi(genre, word)
        if pmi > 0:
            pmi_results[genre].append((word, pmi))
    pmi_results[genre].sort(key=lambda x: x[1], reverse=True)


for genre in genres:
    print(f"top words for {genre}")
    for word, pmi_score in pmi_results[genre][:10]:
        print(f"  {word}: {pmi_score:.2f}")

# ¿Cuáles son los adjetivos/adverbios que están asociados a cada género?




In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

grouped = one_genre_df.groupby("genre")["modifiers"].apply(lambda words: " ".join(sum(words, [])))

genres = grouped.index.tolist()
corpus = grouped.tolist()

tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(corpus)

feature_names = tfidf.get_feature_names_out()
adj_stopwords=["young","old","new"]
top_per_genre = {}
for i, genre in enumerate(genres):
    row = X[i].toarray()[0]
    top_indices = row.argsort()[-15:][::-1]
    top_per_genre[genre] = [(feature_names[j], row[j]) for j in top_indices]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_top_modifiers(genre, top_n=15):

    all_words, all_scores = zip(*top_per_genre[genre][:top_n])


    filtered_words = []
    filtered_scores = []
    for i in range(len(all_words)):
        if all_words[i] not in adj_stopwords:
            filtered_words.append(all_words[i])
            filtered_scores.append(all_scores[i])

    plt.figure(figsize=(8,4))
    sns.barplot(x=filtered_scores, y=filtered_words, orient="h")
    plt.title(f"Top {len(filtered_words)} adjectives of {genre}")
    plt.xlabel("TF-IDF weight")
    plt.tight_layout()
    plt.show()

for genre in genres:
  plot_top_modifiers(genre)

In [ ]:
from wordcloud import WordCloud

def wordcloud_for_genre(genre):
    # Filter out 'young' from the modifiers list
    filtered_modifiers = [mod for mod in one_genre_df[one_genre_df["genre"] == genre]["modifiers"].sum() if mod not in adj_stopwords]
    text = " ".join(filtered_modifiers)
    wc = WordCloud(width=800, height=400, background_color="white").generate(text)
    plt.clf()
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"Adjectives in {genre}")
    plt.show()

for genre in genres:
  wordcloud_for_genre(genre)

# Prediccion de generos

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
import numpy as np

train_df, test_df = train_test_split(multi_genre_df, test_size=0.2, random_state=42)

mlb = MultiLabelBinarizer()
mlb.fit(multi_genre_df["genres"])

train_df["labels"] = mlb.transform(train_df["genres"]).astype(np.float32).tolist()
test_df["labels"] = mlb.transform(test_df["genres"]).astype(np.float32).tolist()

dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "test": Dataset.from_pandas(test_df)
})

In [ ]:
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    texts = examples["synopsis"]
    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=256
    )

encoded_dataset_raw = dataset_dict.map(tokenize_function, batched=True)
encoded_dataset = encoded_dataset_raw.remove_columns(["genres", "synopsis"])
encoded_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/4080 [00:00<?, ? examples/s]

Map:   0%|          | 0/1020 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification,Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(mlb.classes_),
    problem_type="multi_label_classification"
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
from sklearn.metrics import f1_score
from google.colab import drive
drive.mount('/content/drive')


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))  # sigmoid
    preds = (probs > 0.5).astype(int)
    macro_f1 = f1_score(labels, preds, average="macro")
    return {"macro_f1": macro_f1}

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/TP NLP/Entrega Final/Corridas",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


Mounted at /content/drive


/tmp/ipython-input-295420999.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
LAST_CHECKPOINT_PATH = "/content/drive/MyDrive/TP NLP/Entrega Final/Corridas/checkpoint-3400" # Remplazar con tu ruta real

#trainer.train(resume_from_checkpoint=LAST_CHECKPOINT_PATH)
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: uarias (uarias-itba) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Macro F1
1,0.328400,0.305787,0.074040
2,0.282700,0.264464,0.288984
3,0.239400,0.244089,0.455048
4,0.195900,0.240428,0.499139
5,0.155800,0.244608,0.523633


TrainOutput(global_step=2550, training_loss=0.23860172963609882, metrics={'train_runtime': 1121.2779, 'train_samples_per_second': 18.194, 'train_steps_per_second': 2.274, 'total_flos': 2684094206976000.0, 'train_loss': 0.23860172963609882, 'epoch': 5.0})

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

eval=trainer.evaluate()
predictions = trainer.predict(encoded_dataset["test"])
logits = predictions.predictions
true_labels = predictions.label_ids

#save eval as txt
with open('/content/drive/MyDrive/TP NLP/Entrega Final/Resultados/eval.txt', 'w') as f:
    f.write(str(eval))
    f.write('\n')
    f.write(str(predictions))
    f.write('\n')
    f.write(str(mlb.classes_))
    f.write('\n')

print(eval)
probs = 1 / (1 + np.exp(-logits))
preds_binary = (probs > 0.5).astype(int)
print("\n" + "="*60)
print("MATRICES DE CONFUSIÓN BINARIA INDIVIDUALES")
print("="*60)

# Iterar sobre cada clase/género
for i, genre_name in enumerate(mlb.classes_):

    # Obtener las etiquetas reales (columna i) y las predicciones (columna i)
    y_true_genre = true_labels[:, i]
    y_pred_genre = preds_binary[:, i]

    # Crear la Matriz de Confusión 2x2 para este género
    cm = confusion_matrix(y_true_genre, y_pred_genre)

    # Visualizar la matriz (opcional, pero ayuda mucho)
    plt.figure(figsize=(4, 4))
    sns.heatmap(
        cm,
        annot=True,
        fmt='d',
        cmap='Blues',
        xticklabels=['NO', 'SÍ'],
        yticklabels=['NO', 'SÍ']
    )
    plt.title(f'Matriz de Confusión para: {genre_name}')
    plt.ylabel('Etiqueta Verdadera')
    plt.xlabel('Predicción del Modelo')
    plt.savefig(f'/content/drive/MyDrive/TP NLP/Entrega Final/Resultados/confusion_matrix_{genre_name.replace(" ", "_")}.png')
    plt.close()

    print(f"\nMatriz de Confusión para: {genre_name}")
    print(cm)
    print("---------------------------------------")

{'eval_loss': 0.2446083426475525, 'eval_macro_f1': 0.5236327023279026, 'eval_runtime': 13.585, 'eval_samples_per_second': 75.083, 'eval_steps_per_second': 9.422, 'epoch': 5.0}

MATRICES DE CONFUSIÓN BINARIA INDIVIDUALES

Matriz de Confusión para: Action
[[870  38]
 [ 67  45]]
---------------------------------------

Matriz de Confusión para: Adventure
[[912  22]
 [ 61  25]]
---------------------------------------

Matriz de Confusión para: Animation
[[874  22]
 [ 83  41]]
---------------------------------------

Matriz de Confusión para: Comedy
[[655  97]
 [151 117]]
---------------------------------------

Matriz de Confusión para: Crime
[[906  31]
 [ 41  42]]
---------------------------------------

Matriz de Confusión para: Documentary
[[893  32]
 [ 47  48]]
---------------------------------------

Matriz de Confusión para: Drama
[[513 162]
 [136 209]]
---------------------------------------

Matriz de Confusión para: Family
[[880  27]
 [ 63  50]]
-----------------------------------

## Prueba: ejemplo de inferencia para géneros con buena vs mala performance

In [ ]:
import numpy as np
import pandas as pd
# Asegúrate de que encoded_dataset["test"] sea accesible y contenga la columna 'synopsis'
# (o que se pueda mapear de nuevo a la sinopsis original del test_df)

# --- (Asumo que el código anterior ya se ejecutó: predictions, logits, true_labels, preds_binary, mlb) ---

# --- Asumo que tienes el dataset de prueba original disponible con la columna 'synopsis' ---
# Si NO lo tienes, reemplaza esta línea con el dataset que contiene las sinopsis:
test_synopses = encoded_dataset_raw["test"]["synopsis"]
# Si la columna 'synopsis' fue eliminada, necesitarás mapear los índices a tu DataFrame original (test_df)
# En este ejemplo, usaré el índice del dataset.

print("\n" + "="*80)
print("EJEMPLOS DE INFERENCIA POR GÉNERO (Verdadero Positivo o Falso Negativo)")
print("="*80)

# Almacena los índices ya utilizados para no repetir la sinopsis
used_indices = set()

# Iterar sobre cada clase/género para encontrar un ejemplo
for i, genre_name in enumerate(mlb.classes_):

    # 1. Buscar índices donde la etiqueta real para este género es SÍ (1)
    # y que no hayamos usado ya (para mostrar un ejemplo único si es posible)
    is_genre_true = np.where(true_labels[:, i] == 1)[0]

    example_idx = -1

    # Buscar el primer índice donde el género es 1 Y el índice no se haya usado
    for idx in is_genre_true:
        if idx not in used_indices:
            example_idx = idx
            used_indices.add(idx)
            break

    # Si no se encuentra un ejemplo único, al menos encuentra el primero
    if example_idx == -1 and len(is_genre_true) > 0:
        example_idx = is_genre_true[0]

    example_idx = int(example_idx)
    if example_idx != -1:
        # 2. Recuperar la información de la fila
        sinopsis = test_synopses[example_idx] # O test_df.iloc[example_idx]['synopsis']

        # Etiqueta REAL para TODOS los géneros en esta fila
        real_vector = true_labels[example_idx, :]
        real_genres = list(mlb.inverse_transform(real_vector.reshape(1, -1))[0])

        # Predicción para TODOS los géneros en esta fila
        pred_vector = preds_binary[example_idx, :]
        predicted_genres = list(mlb.inverse_transform(pred_vector.reshape(1, -1))[0])

        # 3. Imprimir el resultado
        print(f"\n🎬 GÉNERO EN EXAMEN: {genre_name.upper()}")
        print(f"   - Sinopsis (Fragmento): {sinopsis}...")
        print(f"   - Valor REAL (Todos): {real_genres}")
        print(f"   - Valor PREDECIDO (Todos): {predicted_genres}")

        # Estado del género específico:
        status = ""
        if real_vector[i] == 1 and pred_vector[i] == 1:
            status = "✅ Verdadero Positivo (TP)"
        elif real_vector[i] == 1 and pred_vector[i] == 0:
            status = "❌ Falso Negativo (FN)"

        print(f"   - Estado para {genre_name}: {status}")

    else:
        print(f"\n🎬 GÉNERO EN EXAMEN: {genre_name.upper()}")
        print("   - No se encontraron ejemplos en el conjunto de prueba con etiqueta Real=SÍ.")

print("\n" + "="*80)


EJEMPLOS DE INFERENCIA POR GÉNERO (Verdadero Positivo o Falso Negativo)

🎬 GÉNERO EN EXAMEN: ACTION
   - Sinopsis (Fragmento): Wong Kam-Kwei, a gang boss on the decline, overseeing bathhouses and karaoke bars, falls for a small restaurant owner, Mei. He begins to help her business, and learns that Mei is already in love with Leung, a member of his gang. He’s also like a brother to Kam-Kwei. When a rival gang hoping to steal his territory kills Leung, the only thing left for Kam-Kwei to do now is to protect Mei and avenge Leung....
   - Valor REAL (Todos): ['Action', 'Comedy', 'Crime']
   - Valor PREDECIDO (Todos): ['Action']
   - Estado para Action: ✅ Verdadero Positivo (TP)

🎬 GÉNERO EN EXAMEN: ADVENTURE
   - Sinopsis (Fragmento): Young Dorothy finds herself in a magical world where she makes friends with a lion, a scarecrow and a tin man as they make their way along the yellow brick road to talk with the Wizard and ask for the things they miss most in their lives. The Wicked Witch o

# Longitud y f1-score

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

# ===================================================
# 1. PREPARACIÓN DE DATOS (Asumiendo que ya existen)
# ===================================================
# test_synopses_text: Lista de strings (sinopsis originales del conjunto de prueba)
# true_labels: Matriz NumPy de etiquetas reales (binaria)
# preds_binary: Matriz NumPy de predicciones (binaria)
test_synopses_text = encoded_dataset_raw["test"]["synopsis"]

# 2. CALCULAR LONGITUDES
# Usamos el tokenizador de BERT, pero calcular la longitud en palabras/caracteres es más simple
synopsis_lengths = [len(s.split()) for s in test_synopses_text]
synopsis_lengths = np.array(synopsis_lengths)


# 3. CALCULAR F1 SCORE POR EJEMPLO (Sample F1)
# Calculamos el F1 score para cada fila (eje=1)
# Usamos 'samples' para calcular una métrica por muestra
#sample_f1 = f1_score(true_labels, preds_binary, average='samples', zero_division=0)
from sklearn.metrics import f1_score
import numpy as np

# Asegúrate de que tus matrices de entrada sean arrays de NumPy para iteración
y_true_np = np.asarray(true_labels)
y_pred_np = np.asarray(preds_binary)

# Lista temporal para guardar el F1 score de cada sinopsis
f1_scores_list = []

# Iterar sobre cada fila (muestra)
for i in range(y_true_np.shape[0]):
    # Calcular el F1 score para la i-ésima muestra
    # Usamos average='micro' aquí para calcular el F1 sobre la única muestra (fila)
    # Esto es matemáticamente equivalente a la definición de F1 'sample-wise' para una fila.
    f1_single_sample = f1_score(
        y_true_np[i].reshape(1, -1), # Fila de etiquetas reales
        y_pred_np[i].reshape(1, -1), # Fila de predicciones
        average='micro',
        zero_division=0
    )
    f1_scores_list.append(f1_single_sample)

# Convertir la lista resultante en el array de NumPy que necesitas
sample_f1 = np.array(f1_scores_list)

# VERIFICACIÓN:
print(f"Tipo de sample_f1 después de la iteración: {type(sample_f1)}")
print(f"Longitud de sample_f1: {len(sample_f1)}")
# Esto debe ser <class 'numpy.ndarray'> y longitud 1020.
# ===================================================
# 4. AGRUPAR EN BINS Y CALCULAR EL F1 PROMEDIO
# ===================================================

# Definir los BINS de longitud (puedes ajustar estos límites)
# Ejemplo: 0-50, 51-100, 101-150, 151-200, >200 palabras
bins = [0, 50, 100, 150, 200]
bin_labels = ['0-50', '51-100', '101-150', '151-200']

# Asignar cada ejemplo a un bin
bin_indices = np.digitize(synopsis_lengths, bins[:-1])

# Crear un DataFrame/diccionario para almacenar los resultados por bin
f1_per_bin = {}
num_samples_per_bin = {}

for i, label in enumerate(bin_labels):
    # Encontrar los índices de los ejemplos que caen en este bin
    indices_in_bin = np.where(bin_indices == i + 1)[0]

    if len(indices_in_bin) > 0:
        # Calcular el F1 promedio para los ejemplos en este bin
        avg_f1 = sample_f1[indices_in_bin].mean()
        f1_per_bin[label] = avg_f1
        num_samples_per_bin[label] = len(indices_in_bin)
    else:
        f1_per_bin[label] = 0.0
        num_samples_per_bin[label] = 0

# Convertir a listas para el plotting
bin_names = list(f1_per_bin.keys())
f1_values = list(f1_per_bin.values())


# ===================================================
# 5. CREAR EL HISTOGRAMA (Gráfico de Barras)
# ===================================================

plt.figure(figsize=(10, 6))
# Crear el gráfico de barras del F1 Score promedio por bin de longitud
bars = plt.bar(bin_names, f1_values, color='teal')
#

plt.title('Correlación entre Longitud de Sinopsis y F1-Score promedio (Sample F1)', fontsize=14)
plt.xlabel('Longitud de Sinopsis (en palabras)', fontsize=12)
plt.ylabel('F1-Score Promedio', fontsize=12)
plt.ylim(0, 1.0) # El F1 score va de 0 a 1

# Agregar el número de muestras en cada barra (útil para la interpretación)
for bar, num_samples in zip(bars, num_samples_per_bin.values()):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.02,
             f'N={num_samples}', ha='center', va='bottom', fontsize=10)

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()